# Using FAI to solve Atari environments

# TLDR 

1. In the notebook toolbar click Kernel -> **Restart & Run all**.
2. **Wait a bit**. Usually under 3 minutes in a laptop(1 i7 core) when you are lucky, 10 minutes when you are not.
3. **You should have solved** the **Boxing-v0** Atari environment using a uniform prior and 75 samples per action.
4. There is a **video** of the game played **inside** the ***videos* folder** of this repository.

### Import everything we will need

In [1]:
import numpy as np
from fractalai.policy import GreedyPolicy
from fractalai.model import RandomDiscreteModel
from fractalai.fractalai import FractalAI
from fractalai.environment import AtariEnvironment
from fractalai.monitor import AtariMonitorPolicy

### What we measured

The following table represents the highest scores that we have achieved in the games we tried. Please write us if you try to replicate them, specially if you don't succeed. 

You can use this as a quick cheat sheet to choose parameters that should work.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Game</th>
      <th>FAI Score</th>
      <th>% vs Best AI</th>
      <th>% vs MCTS</th>
      <th>Mean samples / action</th>
      <th>N repeat action</th>
      <th>Time horizon</th>
      <th>Max samples / action</th>
      <th>Max states</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>alien</td>
      <td>19,380</td>
      <td>271.89%</td>
      <td>NaN</td>
      <td>1,190</td>
      <td>5</td>
      <td>25</td>
      <td>2,000</td>
      <td>50</td>
    </tr>
    <tr>
      <th>1</th>
      <td>amidar</td>
      <td>4,306</td>
      <td>194.40%</td>
      <td>NaN</td>
      <td>1,222</td>
      <td>5</td>
      <td>25</td>
      <td>2,000</td>
      <td>50</td>
    </tr>
    <tr>
      <th>2</th>
      <td>assault</td>
      <td>1,280</td>
      <td>17.06%</td>
      <td>NaN</td>
      <td>1,317</td>
      <td>5</td>
      <td>25</td>
      <td>2,000</td>
      <td>50</td>
    </tr>
    <tr>
      <th>3</th>
      <td>asteroids</td>
      <td>76,270</td>
      <td>160.94%</td>
      <td>NaN</td>
      <td>2,733</td>
      <td>2</td>
      <td>20</td>
      <td>5,000</td>
      <td>nan</td>
    </tr>
    <tr>
      <th>4</th>
      <td>beam rider</td>
      <td>2,160</td>
      <td>12.64%</td>
      <td>29.86%</td>
      <td>4,052</td>
      <td>5</td>
      <td>25</td>
      <td>2,000</td>
      <td>100</td>
    </tr>
    <tr>
      <th>5</th>
      <td>boxing</td>
      <td>100</td>
      <td>104.17%</td>
      <td>NaN</td>
      <td>2,027</td>
      <td>3</td>
      <td>30</td>
      <td>2,000</td>
      <td>150</td>
    </tr>
    <tr>
      <th>6</th>
      <td>breakout</td>
      <td>36</td>
      <td>7.98%</td>
      <td>8.87%</td>
      <td>5,309</td>
      <td>5</td>
      <td>30</td>
      <td>20,000</td>
      <td>150</td>
    </tr>
    <tr>
      <th>7</th>
      <td>centipede</td>
      <td>529,355</td>
      <td>4,405.05%</td>
      <td>NaN</td>
      <td>1,960</td>
      <td>1</td>
      <td>20</td>
      <td>6,000</td>
      <td>100</td>
    </tr>
    <tr>
      <th>8</th>
      <td>double dunk</td>
      <td>20</td>
      <td>400.00%</td>
      <td>NaN</td>
      <td>5,327</td>
      <td>3</td>
      <td>50</td>
      <td>8,000</td>
      <td>nan</td>
    </tr>
    <tr>
      <th>9</th>
      <td>enduro</td>
      <td>471</td>
      <td>28.05%</td>
      <td>59.77%</td>
      <td>nan</td>
      <td>5</td>
      <td>15</td>
      <td>2,000</td>
      <td>50</td>
    </tr>
    <tr>
      <th>10</th>
      <td>ice hockey</td>
      <td>52</td>
      <td>5,200.00%</td>
      <td>NaN</td>
      <td>12,158</td>
      <td>3</td>
      <td>50</td>
      <td>20,000</td>
      <td>250</td>
    </tr>
    <tr>
      <th>11</th>
      <td>ms pacman</td>
      <td>58,521</td>
      <td>372.91%</td>
      <td>NaN</td>
      <td>5,129</td>
      <td>10</td>
      <td>20</td>
      <td>20,000</td>
      <td>250</td>
    </tr>
    <tr>
      <th>12</th>
      <td>qbert</td>
      <td>35,750</td>
      <td>189.66%</td>
      <td>189.66%</td>
      <td>2,728</td>
      <td>5</td>
      <td>20</td>
      <td>5,000</td>
      <td>nan</td>
    </tr>
    <tr>
      <th>13</th>
      <td>seaquest</td>
      <td>3,180</td>
      <td>7.56%</td>
      <td>97.64%</td>
      <td>6,252</td>
      <td>5</td>
      <td>25</td>
      <td>20,000</td>
      <td>250</td>
    </tr>
    <tr>
      <th>14</th>
      <td>space invaders</td>
      <td>3,605</td>
      <td>80.29%</td>
      <td>153.14%</td>
      <td>4,261</td>
      <td>2</td>
      <td>35</td>
      <td>6,000</td>
      <td>nan</td>
    </tr>
    <tr>
      <th>15</th>
      <td>tennis</td>
      <td>16</td>
      <td>177.78%</td>
      <td>NaN</td>
      <td>2,437</td>
      <td>4</td>
      <td>30</td>
      <td>5,000</td>
      <td>nan</td>
    </tr>
    <tr>
      <th>16</th>
      <td>video pinball</td>
      <td>496,681</td>
      <td>64.64%</td>
      <td>NaN</td>
      <td>1,083</td>
      <td>2</td>
      <td>30</td>
      <td>5,000</td>
      <td>nan</td>
    </tr>
    <tr>
      <th>17</th>
      <td>wizard of wor</td>
      <td>93,090</td>
      <td>785.44%</td>
      <td>NaN</td>
      <td>2,229</td>
      <td>4</td>
      <td>35</td>
      <td>5,000</td>
      <td>nan</td>
    </tr>
  </tbody>
</table>

## Interpreting the parameter choice

The agent relies on four parmeters:

- **Fixed steps**: Although this parameter actually depends on the Environment, we can use it to manually set the frecuancy at which the Agent will play. Taking more consecutive actions allows for exploring further in the future at the cost of less reaction time.


- **Time Horizon**: This value represents "how far we need to look into the future when taking an action". A useful rule of thumb is **Time Horiozon = Nt / Fixed steps**, where **Nt** is the number of frames that it takes the agent to loose one life, (die) since the moment it performs the actions that inevitably leads to its death. 


- **Max states**: This is the maximum number of states that can be part of the Swarm. This number is related to "how thick" we want the resulting causal cone to be. The algorithm will try to use the maximum number of states possible unless it detects it is wasting computation.


- **Max samples**: This is the maximum number of times that we can sample an action when using a Swarm to build a causal cone. It is a superior bound, and the algorithm will try to use less samples to meet the defined **time horizon**. It is a nice way to limit how fast you need to take an action. A reasonable value could be **max walkers** \* **time horizon** \* ***N***, being ***N=5*** a number that works well in Atari games, but it depends on what you are trying to accomplish.


- **Mean samples**: You cannot directly set this parameter. It is the mean number of samples taken each state. In and ideal case it would be **Max states \* Time horizon** samples. If its lower it means that we are not having any trouble sampling the state space, but if its higher it means that the states tend to die and more computation has been be required. 


## Practical examples

 ### Minimal Pacman

Now we will tune the Agent to get a decent score on MsPacman using the minimum amount of computational resources possible. We will deliberately set a very small amount of computational resources for calculating an action.

Doing that we want to address concerns about edge cases of the theory, by showing how the algorithm performs when the size of the swarm Swarm is very little with respect to the size of the state space.

In order to do so, we can give the parameters the following values:

#### Environment Parameters

```python
name = "MsPacman-v0"
render=True # It is funnier if the game is displayed on the screen
clone_seeds = True # This will speed things up a bit
max_steps = 1e6 # Play until the game is finished.
skip_frames = 80 # The Agent cannot do anything anyway, so its faster if we skip some frames at the begining
n_fixed_steps=5 # Atari games run at 20 fps, so taking 4 actions per seconds is enough to finish the first level```


#### FAI parameters

```python
max_samples=300 # Let see how well it can perform using 300 samples per step
max_states=15 # Let's set a really small number to make everthing faster
time_horizon=10 # 50 frames should be enough to realise you have been eaten by a ghost```

With these parameters we are aiming for 150 samples per step, saving up to another 150 samples in case the agent runs into trouble. Using such a low number of samples will mean that the performance could vary widely among different runs, but in our tests, this agent was capable of finishing the first level most of the times.

If you want to get better scores, just increase the values of the parameters accordingly.

## Available atari games

In [9]:
>>> import atari_py as ap
>>> game_list = ap.list_games()
>>> print(sorted(game_list))
<<< ['adventure',
     'air_raid',
     'alien',
     'amidar',
     'assault',
     'asterix',
     'asteroids',
     'atlantis',
     'bank_heist',
     'battle_zone',
     'beam_rider',
     'berzerk',
     'bowling',
     'boxing',
     'breakout',
     'carnival',
     'centipede',
     'chopper_command',
     # ...
     ]

SyntaxError: invalid syntax (<ipython-input-9-cf616e3c9815>, line 4)

 ### Minimal Boxing

We will follow the same principles as in the former example, but this time aiming to solve the game Boxing. This probably turns out to be the least complex Atari game, and it can be solved with very little amount of computation involved.

A funny thing about this game is the only one with the same reward scheme that is supposed to be solved in the minimum amount of time possible. But without stating it explicitely.

A consequence of that, the agent sometimes just starts avoiding the opponent after achieving 99 points, because it considers a terminal state as a death, and it tries to keep himself alive until the time runs out.

There are many ways of addressing this problem, but they are out of the scope of this notebook.

#### Environment Parameters

In [10]:
name = "MsPacman-v0"
render=True # It is funnier if the game is displayed on the screen
clone_seeds = True # This will speed things up a bit
max_steps = 1e6 # Play until the game is finished.
skip_frames = 80 # The Agent cannot do anything anyway, so its faster if we skip some frames at the begining
n_fixed_steps=5 # Atari games run at 20 fps, so taking 4 actions per seconds is enough to finish the first level

#### FAI parameters

In [11]:
max_samples=300 # Let see how well it can perform using 300 samples per step
max_states=15 # Let's set a really small number to make everthing faster
time_horizon=10 # 50 frames should be enough to realise you have been eaten by a ghost

### Creating the agent

In [12]:
# we will use a random prior
env =  AtariEnvironment(name=name, clone_seeds=clone_seeds)
model = RandomDiscreteModel(env.env.action_space.n)
greedy = GreedyPolicy(env=env, model=model)

fractal = FractalAI(policy=greedy, max_samples=max_samples, max_states=max_states,
                    time_horizon=time_horizon, n_fixed_steps=n_fixed_steps)

In [5]:
# Runs faster if you are not recording, uncomment if you want to run a simulation.
# fractal.evaluate(render=render, max_steps=max_steps, skip_frames=skip_frames)

## Record a video

This will make everything a bit slower, but you will be recording the game with an OpenAI monitor.

In [13]:
directory="videos_monte" # Folder where you can save the video.
force = True # Override previous data

In [14]:
monitor = AtariMonitorPolicy(policy=fractal, directory=directory, force=force)
monitor.record_video(skip_frames=skip_frames)

KeyboardInterrupt: 

### Check the video folder to watch what you recorded!

## We will really appreciate your feedback

In [15]:
monitor.monitor.env.close() # it should close when the game finishes, but just to be sure you can run this